In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from utils_data import x_y_split , data_splitter
from utils_lgbm import lgbm_train_val_test_split_original
from lgbm_data_prep import LgbmDataPrep
from sklearn.pipeline import Pipeline, FeatureUnion
import numpy as np
import pandas as pd

In [13]:
# split to train test
# split to X, y
# features pipe separate from the targets pipe

### Split To Original train and test

In [14]:
x,v,t = lgbm_train_val_test_split_original()

### split to X, y 

In [15]:
X, y = x_y_split(x,'anomaly')

In [16]:
### data splitter - for multimodel
splits = data_splitter(X,5)

In [17]:
len(splits)

5

In [18]:
feature_pipe =  LgbmDataPrep().get_features_pipeline()
targets_pipe = LgbmDataPrep().get_targets_pipeline()

In [23]:
feature_pipe

FeatureUnion(transformer_list=[('iter_0',
                                Pipeline(steps=[('data_splitter_0',
                                                 DataSplitter(num_splits=5,
                                                              part_num=0)),
                                                ('drop_changepoint',
                                                 DropColumnTransformer(columns_to_drop=['changepoint'])),
                                                ('smooth', SmoothCurve()),
                                                ('scaler', StandardScaler()),
                                                ('df_creator',
                                                 CreateStatsDataframe())])),
                               ('iter_1',
                                Pipeline(steps=[('data_splitter_1',
                                                 DataSplitter(num_spl...
                                                 DropColumnTransformer(columns_to_drop=['changepoint'])),
                                                ('smooth', SmoothCurve()),
                                                ('scaler', StandardScaler()),
                                                ('df_creator',
                                                 CreateStatsDataframe())])),
                               ('iter_4',
                                Pipeline(steps=[('data_splitter_4',
                                                 DataSplitter(num_splits=5,
                                                              part_num=4)),
                                                ('drop_changepoint',
                                                 DropColumnTransformer(columns_to_drop=['changepoint'])),
                                                ('smooth', SmoothCurve()),
                                                ('scaler', StandardScaler()),
                                                ('df_creator',
                                                 CreateStatsDataframe())]))])

In [21]:
feature_pipe.fit(X)
# targets_pipe.transform(y)

0
1
2
3
4


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 2533 and the array at index 4 has size 2535

In [ ]:
[feature_pipe]*3

In [ ]:
import pandas as pd
from typing import List
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

class DataSplitter(BaseEstimator, TransformerMixin):
    def __init__(self, num_splits: int,part_num):
        self.num_splits = num_splits
        self.part_num = part_num

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        """
        Splits a dataframe to multiple splits determined by num_splits argument

        Args:
            X (pd.DataFrame): The data set to split by the index (not shuffled)

        Returns:
            List[pd.DataFrame]: list that contains the dataframe splits
        """
        num_rows_per_part = len(X) // self.num_splits
        parts = []
        for i in range(self.num_splits):
            if i < self.num_splits - 1:
                part = X.iloc[i * num_rows_per_part: (i + 1) * num_rows_per_part]
            else:
                # For the last part, include the remaining rows
                part = X.iloc[i * num_rows_per_part:]
            parts.append(part)
        if self.part_num is not None:
            print(self.part_num)
            parts = parts[self.part_num]
        return parts


In [ ]:
data_splitter = Pipeline(
            [(f'data_part_{i}' ,DataSplitter(num_splits=5,part_num=i)) for i in range(5)]
            )

In [ ]:
# Example usage:
# Create an instance of the transformer
transformer = DataSplitter(num_splits=3,part_num=2)

# Apply the transformer to your data
data_splits = transformer.fit_transform(x)

In [ ]:
data_splits

### examples

In [ ]:
# 1. load the datasets - by config
#   1.1 split for train test (if validation set is needed it will this function again)
#   1.2 - split the data for each model
# 2. data pipelines - for each model - create data pipeline for each datafreame in a loop, then append the model
# 3. model as a transformer
#  4. aggregate the results to one df - (feature union) - example in run models.ipynb
# 5. model evaluation
# 6. model selection


# pipeline.fit_transform(X_train,y_train)
#pipeline.transform(X_test)


#### the implemetation shuold be close to this:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Create synthetic data
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define transformers
transformer_list = [
    ('scaler', StandardScaler()),  # Standardize features
    ('pca', PCA(n_components=10))  # Reduce dimensionality
]

# Define a FeatureUnion combining the transformers
feature_union = FeatureUnion(transformer_list)

# Define the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Create a pipeline
pipeline = Pipeline([
    ('features', feature_union),  # Apply the feature union
    ('classifier', model)  # Apply the classifier
])

# Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


In [ ]:
full_pipeline

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
import pandas as pd

# Sample data
data = pd.DataFrame({
    'feature1': [1, 2, 3],
    'feature2': [4, 5, 6],
    'feature3': ['A', 'B', 'C']
})

class DataFilter(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.columns]

# Define pipelines for each branch of the FeatureUnion
pipeline1 = Pipeline([
    ('filter1', DataFilter(['feature1', 'feature2'])),
    ('imputer1', SimpleImputer(strategy='median')),
    ('scaler1', StandardScaler())
])

pipeline2 = Pipeline([
    ('filter2', DataFilter(['feature3'])),
    ('imputer2', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine the pipelines using FeatureUnion
full_pipeline = FeatureUnion(transformer_list=[
    ('pipeline1', pipeline1),
    ('pipeline2', pipeline2)
])

# Fit and transform the data using the full pipeline
transformed_data = full_pipeline.fit_transform(data)

# The transformed_data now contains the transformed data for each input dataframe, filtered differently for each branch


In [ ]:
full_pipeline